In [1]:
!pip install -q timm
# timm provides all the transformers here 
#timm = pre-trained models, torch = DL ka engine

import timm, torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
path = "/kaggle/input/vegetable-image-dataset/Vegetable Images/train"
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

train = datasets.ImageFolder(path, transform=transform) # Load images from folders (folder = class)
loader = DataLoader(train, batch_size=32, shuffle=True)
classes = train.classes
#Image ko 224×224 karna image ko tensor mai convert karna and normalizing pixels with imagenet images mean and stuff

In [5]:
model = timm.create_model("deit_base_patch16_224", pretrained = True, num_classes=len(classes)).cuda()

# Phelle sarre layers in VIT ko lock karna imp hai -----

In [6]:
for param in model.parameters():
    param.requires_grad=False
    

# .blocks se list kiye ki kitne layers hai and [-2:] sirf last two layers ko choose kiya 
and unko  requires_grad =True kardiyye takii voh training hosakke

In [7]:
for block in model.blocks[-2:]:
    for param in block.parameters():
        param.requires_grad=True

# Classification head ko bhi unfreeze karna

In [8]:
for param in model.head.parameters():
    param.requires_grad = True


In [10]:
total = sum(1 for _ in model.parameters())
trainable = sum(1 for p in model.parameters() if p.requires_grad)

print(f"\nTrainable parameters: {trainable} / {total}")



Trainable parameters: 26 / 152


In [13]:
opt = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)
loss_fn = nn.CrossEntropyLoss() 

In [15]:
for epoch in range(5):
    for x, y in loader:
        x, y = x.cuda(), y.cuda()
        opt.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
    print(f"Epoch {epoch+1} done")


Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done


In [16]:
torch.save(model.state_dict(), "vit_custom_head.pth")

In [17]:
test_path = "/kaggle/input/vegetable-image-dataset/Vegetable Images/test"

test_data = datasets.ImageFolder(test_path, transform=transform)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
model.eval()
correct = 0
total = 0

with torch.no_grad():   # gradient ki zarurat nahi
    for x, y in test_loader:
        x, y = x.cuda(), y.cuda()
        out = model(x)              # forward pass
        preds = out.argmax(dim=1)   # sabse high score wali class
        correct += (preds == y).sum().item()
        total += y.size(0)

accuracy = correct / total
print("Partial Fine-tuning Accuracy:", accuracy)


Partial Fine-tuning Accuracy: 1.0


In [ ]:
UNet
UNet++

SAM V2 for segmentation

UNet
UNet++
SAM V2 for segmentation